# Tutorial de Web Scraping: Tesouro Direto

Este notebook demonstra como extrair dados de títulos do Tesouro Direto usando web scraping com Python. Vamos utilizar as bibliotecas `requests` para fazer requisições HTTP e `BeautifulSoup` para analisar o HTML.

## 1. Instalando as bibliotecas necessárias

Primeiro, vamos instalar as bibliotecas que precisaremos para este tutorial:

In [ ]:
#!pip install requests beautifulsoup4 pandas matplotlib

## 2. Importando as bibliotecas

In [14]:
!pip install beautifulsoup4 matplotlib

import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------ --------------------- 3.7/8.1 MB 21.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 22.7 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 25.3 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl (71 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 3. Obtendo os dados do Tesouro Direto

Para extrair os dados do Tesouro Direto, precisamos primeiro identificar a URL correta. O site oficial do Tesouro Direto é https://www.tesourodireto.com.br/, mas vamos precisar acessar a API que fornece os dados atualizados dos títulos.

In [15]:
# Configurando cabeçalhos para simular um navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.tesourodireto.com.br/'
}

### 3.1 Abordagem via API do Tesouro Direto

O Tesouro Direto disponibiliza seus dados através de uma API que retorna os dados em formato JSON. Vamos acessar essa API para obter os títulos disponíveis.

In [16]:
# URL da API que fornece os títulos disponíveis
url_api = "https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json"

# Fazendo a requisição
response = requests.get(url_api, headers=headers)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    print("Requisição bem-sucedida!")
    data = response.json()
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)

Requisição bem-sucedida!


### 3.2 Explorando os dados retornados

In [17]:
import json
# Analisando a estrutura dos dados
if 'data' in response.json():
    # Algumas vezes os dados estão dentro de uma chave 'data'
    print("Estrutura dos dados:")
    print(json.dumps(response.json()['data'].keys(), indent=2))
else:
    # Caso contrário, analisamos as chaves do JSON diretamente
    print("Estrutura dos dados:")
    print(json.dumps(list(response.json().keys()), indent=2))

Estrutura dos dados:
[
  "responseStatus",
  "responseStatusText",
  "statusInfo",
  "response"
]


In [18]:
# Vamos tentar acessar os títulos - adaptando conforme a estrutura real dos dados
try:
    if 'response' in data:
        titulos = data['response']['TrsrBdTradgList']
    elif 'TrsrBdTradgList' in data:
        titulos = data['TrsrBdTradgList']
    else:
        # Tentando outras possíveis estruturas
        for key in data.keys():
            if isinstance(data[key], list) and len(data[key]) > 0:
                titulos = data[key]
                break
    
    # Exibindo o primeiro título para entendermos a estrutura
    print("\nExemplo de um título:")
    print(json.dumps(titulos[0], indent=2))
    
    # Contando quantos títulos temos
    print(f"\nTotal de títulos disponíveis: {len(titulos)}")
    
except Exception as e:
    print(f"Erro ao processar os dados: {e}")
    print("Estrutura completa dos dados:")
    print(json.dumps(data, indent=2)[:1000]) # Limita para não mostrar tudo


Exemplo de um título:
{
  "TrsrBd": {
    "cd": 170,
    "nm": "Tesouro IPCA+ 2026",
    "featrs": "T\u00edtulo p\u00f3s-fixado, uma vez que parte do seu rendimento acompanha a varia\u00e7\u00e3o da taxa de infla\u00e7\u00e3o (IPCA).\r\n",
    "mtrtyDt": "2026-08-15T00:00:00",
    "minInvstmtAmt": 0.0,
    "untrInvstmtVal": 0.0,
    "invstmtStbl": "Aumenta o poder de compra do seu dinheiro, pois seu rendimento \u00e9 composto por uma taxa de juros + a varia\u00e7\u00e3o da infla\u00e7\u00e3o (IPCA). \u00c9 mais interessante para quem pode deixar o dinheiro render at\u00e9 o vencimento do investimento, pois n\u00e3o paga juros semestrais. Em caso de resgate antecipado, o Tesouro Nacional garante sua recompra pelo seu valor de mercado.\r\n",
    "semiAnulIntrstInd": false,
    "rcvgIncm": "Indicado para aqueles que querem realizar investimentos de longo prazo.\r\n",
    "anulInvstmtRate": 0.0,
    "anulRedRate": 9.22,
    "minRedQty": 0.01,
    "untrRedVal": 4038.15,
    "minRedVal": 40

## 4. Processando os dados e criando um DataFrame

Vamos criar um DataFrame com os dados relevantes de cada título.

In [19]:
# Função para extrair informações relevantes dos títulos
def extrair_info_titulos(titulos):
    dados = []
    
    for titulo in titulos:
        try:
            # Adaptando conforme a estrutura real dos dados
            info = {
                'Nome': titulo.get('TrsrBd', {}).get('nm', ''),
                'Vencimento': titulo.get('TrsrBd', {}).get('mtrtyDt', ''),
                'Taxa de Rendimento': titulo.get('anulInvstmtRate', ''),
                'Preço Unitário': titulo.get('untrInvstmtVal', ''),
                'Investimento Mínimo': titulo.get('minInvstmtAmt', ''),
                'Tipo': titulo.get('TrsrBd', {}).get('tp', {}).get('nm', '')
            }
            dados.append(info)
        except Exception as e:
            print(f"Erro ao processar título: {e}")
    
    return dados

In [20]:
# Extraindo as informações e criando o DataFrame
try:
    dados_titulos = extrair_info_titulos(titulos)
    df_titulos = pd.DataFrame(dados_titulos)
    
    # Convertendo colunas numéricas
    for col in ['Taxa de Rendimento', 'Preço Unitário', 'Investimento Mínimo']:
        try:
            df_titulos[col] = pd.to_numeric(df_titulos[col].str.replace(',', '.'), errors='coerce')
        except:
            pass
    
    # Exibindo o DataFrame
    display(df_titulos)
    
except Exception as e:
    print(f"Erro ao criar DataFrame: {e}")
    
    # Alternativa: tentar uma abordagem mais genérica
    print("\nTentando abordagem alternativa...")
    
    # Criar DataFrame com todos os campos disponíveis
    df_titulos = pd.json_normalize(titulos)
    display(df_titulos.head())

,Nome,Vencimento,Taxa de Rendimento,Preço Unitário,Investimento Mínimo,Tipo
0,Tesouro IPCA+ 2026,2026-08-15T00:00:00,NaN,NaN,NaN,
1,Tesouro IPCA+ 2029,2029-05-15T00:00:00,NaN,NaN,NaN,
2,Tesouro IPCA+ 2035,2035-05-15T00:00:00,NaN,NaN,NaN,
3,Tesouro IPCA+ 2040,2040-08-15T00:00:00,NaN,NaN,NaN,
4,Tesouro IPCA+ 2045,2045-05-15T00:00:00,NaN,NaN,NaN,
5,Tesouro IPCA+ 2050,2050-08-15T00:00:00,NaN,NaN,NaN,
6,Tesouro Prefixado com Juros Semestrais 2027,2027-01-01T00:00:00,NaN,NaN,NaN,
7,Tesouro Prefixado com Juros Semestrais 2029,2029-01-01T00:00:00,NaN,NaN,NaN,
8,Tesouro Prefixado com Juros Semestrais 2031,2031-01-01T00:00:00,NaN,NaN,NaN,
9,Tesouro Prefixado com Juros Semestrais 2033,2033-01-01T00:00:00,NaN,NaN,NaN,


## 5. Abordagem alternativa: Web Scraping direto do site

Se a abordagem da API não funcionar corretamente, podemos tentar fazer web scraping diretamente do site do Tesouro Direto.

In [ ]:
# URL da página principal do Tesouro Direto
url_site = "https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm"

# Fazendo a requisição
response_site = requests.get(url_site, headers=headers)

# Verificando se a requisição foi bem-sucedida
if response_site.status_code == 200:
    print("Requisição ao site bem-sucedida!")
    soup = BeautifulSoup(response_site.content, 'html.parser')
else:
    print(f"Erro na requisição ao site: {response_site.status_code}")

In [ ]:
# Localizando a tabela de títulos
try:
    tabelas = soup.find_all('table')
    print(f"Número de tabelas encontradas: {len(tabelas)}")
    
    # Se houver tabelas, vamos tentar extrair os dados da primeira
    if tabelas:
        # Extraindo os cabeçalhos
        headers = []
        header_row = tabelas[0].find('thead').find('tr')
        if header_row:
            headers = [th.get_text(strip=True) for th in header_row.find_all('th')]
        
        # Extraindo os dados das linhas
        rows = []
        for tr in tabelas[0].find('tbody').find_all('tr'):
            row = [td.get_text(strip=True) for td in tr.find_all('td')]
            rows.append(row)
        
        # Criando o DataFrame
        df_site = pd.DataFrame(rows, columns=headers)
        display(df_site)
    else:
        print("Nenhuma tabela encontrada na página.")
        
        # Tentando encontrar os dados em formato estruturado no JavaScript da página
        scripts = soup.find_all('script')
        dados_encontrados = False
        
        for script in scripts:
            if script.string and 'treasuryBondsTrades' in script.string:
                print("Dados encontrados no JavaScript da página!")
                dados_encontrados = True
                # Extrair dados do JavaScript usando regex
                pattern = r'treasuryBondsTrades\s*=\s*(\[.*?\]);'
                match = re.search(pattern, script.string, re.DOTALL)
                if match:
                    json_data = match.group(1)
                    dados_js = json.loads(json_data)
                    df_js = pd.json_normalize(dados_js)
                    display(df_js.head())
                break
        
        if not dados_encontrados:
            print("Não foi possível encontrar os dados dos títulos na página.")
            
except Exception as e:
    print(f"Erro ao extrair dados da tabela: {e}")

## 6. Analisando os dados

In [ ]:
# Função para formatar os dados de forma adequada
def formatar_dataframe(df):
    try:
        # Cópia do DataFrame para não modificar o original
        df_formatado = df.copy()
        
        # Convertendo valores monetários para float
        colunas_monetarias = [col for col in df.columns if 'valor' in col.lower() or 'preço' in col.lower() or 'investimento' in col.lower()]
        for col in colunas_monetarias:
            if col in df.columns:
                df_formatado[col] = df_formatado[col].astype(str).str.replace('R$', '').str.replace('.', '').str.replace(',', '.').astype(float)
        
        # Convertendo percentuais para float
        colunas_percentuais = [col for col in df.columns if 'taxa' in col.lower() or 'rentabilidade' in col.lower() or '%' in col.lower()]
        for col in colunas_percentuais:
            if col in df.columns:
                df_formatado[col] = df_formatado[col].astype(str).str.replace('%', '').str.replace(',', '.').astype(float)
        
        # Convertendo datas
        colunas_data = [col for col in df.columns if 'data' in col.lower() or 'vencimento' in col.lower()]
        for col in colunas_data:
            if col in df.columns:
                try:
                    df_formatado[col] = pd.to_datetime(df_formatado[col], format='%d/%m/%Y', errors='coerce')
                except:
                    pass
        
        return df_formatado
    
    except Exception as e:
        print(f"Erro ao formatar DataFrame: {e}")
        return df

In [ ]:
# Tentando formatar o DataFrame obtido
try:
    if 'df_titulos' in locals():
        df_formatado = formatar_dataframe(df_titulos)
        display(df_formatado)
    elif 'df_site' in locals():
        df_formatado = formatar_dataframe(df_site)
        display(df_formatado)
    elif 'df_js' in locals():
        df_formatado = formatar_dataframe(df_js)
        display(df_formatado)
    else:
        print("Nenhum DataFrame disponível para formatação.")
except Exception as e:
    print(f"Erro ao processar DataFrame: {e}")

## 7. Visualizando os dados

In [ ]:
# Configurando o estilo dos gráficos
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

In [ ]:
# Visualizando as taxas de rendimento por tipo de título
try:
    if 'df_formatado' in locals():
        # Identificando colunas relevantes
        col_tipo = next((col for col in df_formatado.columns if 'tipo' in col.lower()), None)
        col_taxa = next((col for col in df_formatado.columns if 'taxa' in col.lower()), None)
        
        if col_tipo and col_taxa:
            plt.figure(figsize=(12, 6))
            df_formatado.groupby(col_tipo)[col_taxa].mean().sort_values().plot(kind='barh', color='skyblue')
            plt.title('Taxa Média de Rendimento por Tipo de Título')
            plt.xlabel('Taxa de Rendimento (%)')
            plt.tight_layout()
            plt.show()
        else:
            print("Colunas necessárias não encontradas para visualização.")
    else:
        print("DataFrame formatado não disponível.")
except Exception as e:
    print(f"Erro ao criar visualização: {e}")

In [ ]:
# Visualizando a relação entre vencimento e taxa de rendimento
try:
    if 'df_formatado' in locals():
        # Identificando colunas relevantes
        col_vencimento = next((col for col in df_formatado.columns if 'vencimento' in col.lower()), None)
        col_taxa = next((col for col in df_formatado.columns if 'taxa' in col.lower()), None)
        col_tipo = next((col for col in df_formatado.columns if 'tipo' in col.lower()), None)
        
        if col_vencimento and col_taxa:
            plt.figure(figsize=(12, 6))
            if col_tipo:
                for tipo, grupo in df_formatado.groupby(col_tipo):
                    plt.scatter(grupo[col_vencimento], grupo[col_taxa], label=tipo, alpha=0.7)
                plt.legend()
            else:
                plt.scatter(df_formatado[col_vencimento], df_formatado[col_taxa], alpha=0.7, color='blue')
            
            plt.title('Relação entre Data de Vencimento e Taxa de Rendimento')
            plt.xlabel('Data de Vencimento')
            plt.ylabel('Taxa de Rendimento (%)')
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        else:
            print("Colunas necessárias não encontradas para visualização.")
    else:
        print("DataFrame formatado não disponível.")
except Exception as e:
    print(f"Erro ao criar visualização: {e}")

## 8. Salvando os dados em arquivo CSV

In [ ]:
# Salvando os dados obtidos em um arquivo CSV
try:
    if 'df_formatado' in locals():
        # Obtendo a data atual para incluir no nome do arquivo
        data_atual = datetime.now().strftime("%Y-%m-%d")
        nome_arquivo = f"tesouro_direto_{data_atual}.csv"
        
        # Salvando o DataFrame em CSV
        df_formatado.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')
        print(f"Dados salvos com sucesso no arquivo '{nome_arquivo}'")
    else:
        print("Nenhum DataFrame formatado disponível para salvar.")
except Exception as e:
    print(f"Erro ao salvar arquivo CSV: {e}")

## 9. Conclusão

Neste tutorial, aprendemos como extrair dados do Tesouro Direto usando web scraping com Python. Utilizamos duas abordagens:

1. Através da API do Tesouro Direto, que fornece os dados em formato JSON
2. Através do web scraping direto do site, usando BeautifulSoup para analisar o HTML

Processamos os dados obtidos, criamos visualizações e salvamos os resultados em um arquivo CSV.

### Observações importantes:

- A estrutura dos dados no site do Tesouro Direto pode mudar ao longo do tempo, o que pode exigir ajustes no código.
- Este tutorial é apenas para fins educacionais. Sempre verifique os termos de uso do site antes de fazer web scraping.
- Para uso em produção, considere utilizar uma API oficial, se disponível, em vez de web scraping.
- Os dados extraídos estão sujeitos à precisão da fonte e do processo de extração.

## 10. Recursos adicionais

- [Site oficial do Tesouro Direto](https://www.tesourodireto.com.br/)
- [Documentação do BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Documentação do Pandas](https://pandas.pydata.org/docs/)
- [Documentação do Matplotlib](https://matplotlib.org/stable/contents.html)